<b>Note: This Jupyter Notebook is associated with the article [The Bear Put Spread Explained (and How to Trade The Options Strategy with Alpaca)](https://alpaca.markets/learn/bear-put-spread).</b>

# Step 1: Setting Up the Environment and Trade Parameters

In [ ]:
# Install or upgrade the package `alpaca-py` and import it
!python3 -m pip install --upgrade alpaca-py

import pandas as pd
import numpy as np
from scipy.stats import norm
import time
from scipy.optimize import brentq
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from typing import Any, Dict, List, Optional, Tuple

import alpaca
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient, StockLatestTradeRequest
from alpaca.data.requests import StockBarsRequest, OptionLatestQuoteRequest, OptionSnapshotRequest
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import (
    MarketOrderRequest,
    GetOptionContractsRequest,
    MarketOrderRequest,
    OptionLegRequest,
    ClosePositionRequest,
)
from alpaca.trading.enums import (
    AssetStatus,
    OrderSide,
    OrderClass,
    OrderStatus,
    TimeInForce,
    ContractType,
)

In [ ]:
# API_KEY = "Alpaca's Trading API Key (Paper Account)"
# API_SECRET = "Alpaca's Trading API Secret Key (Paper Account)"

# A safe approach to setting up API credentials for Alpaca (Assume you run this notebook in Google Colab)
# Add your key to Colab Secrets. Add your API key to the Colab Secrets manager to securely store it
from google.colab import userdata
API_KEY = userdata.get('ALPACA_API_KEY')
API_SECRET = userdata.get('ALPACA_SECRET_KEY')
BASE_URL = None
## We use paper environment for this example (Please do not modify this. This example is for paper trading only).
PAPER = True

# Initialize Alpaca clients
trade_client = TradingClient(api_key=API_KEY, secret_key=API_SECRET, paper=PAPER, url_override=BASE_URL)
option_historical_data_client = OptionHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET, url_override=BASE_URL)
stock_data_client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

# Step 2: Calculations for Implied Volatility and Option Greeks with Black-Scholes model

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type):
    
    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:
        
        print('Option price is close to intrinsic value; implied volatility is near zero.')
        return 0.0
    
    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None


def calculate_greeks(option_price, strike_price, expiration, underlying_price, risk_free_rate, option_type):
    T = (expiration - pd.Timestamp.now()).days / 365 # It is unconventional, but some use 225 days (# of annual trading days) in replace of 365 days
    T = max(T, 1e-6)  # Set minimum T to avoid zero
    
    if T == 1e-6:
        print('Option has expired or is expiring now; setting Greeks based on intrinsic value.')
        if option_type == 'put':
            delta = -1.0 if underlying_price < strike_price else 0.0
        else:
            delta = 1.0 if underlying_price > strike_price else 0.0
        gamma = 0.0
        theta = 0.0
        vega = 0.0
        return delta, gamma, theta, vega
    
    # Calculate IV
    IV = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)

    if IV is None or IV == 0.0:
        print('Implied volatility could not be determined, skipping Greek calculations.')
        return None
    
    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * IV ** 2) * T) / (IV * np.sqrt(T))
    d2 = d1 - IV * np.sqrt(T) # d2 for Theta calculation
    # Calculate Delta
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    # Calculate Gamma
    gamma = norm.pdf(d1) / (underlying_price * IV * np.sqrt(T))
    # Calculate Vega
    vega = underlying_price * np.sqrt(T) * norm.pdf(d1) / 100
    # Calculate Theta
    if option_type == 'call':
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            - (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(d2))
        )
    else:
        theta = (
            - (underlying_price * norm.pdf(d1) * IV) / (2 * np.sqrt(T))
            + (risk_free_rate * strike_price * np.exp(-risk_free_rate * T) * norm.cdf(-d2))
        )
    # Convert annualized theta to daily theta
    theta /= 365
    
    return delta, gamma, theta, vega

# Step 3: Choosing the Right Strike Price, Expiration, and Other Key Factors

**Step 3-A:** Pick Your Strategy Inputs – Right Strike Price and Expiration  
**Step 3-B:** Pick Your Strategy Inputs – Risk Management and Position Sizing

In [ ]:
# Select the underlying stock
underlying_symbol = 'AAPL'

# Set the timezone
timezone = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(timezone).date()

# Define a 6% range around the underlying price 
STRIKE_RANGE = 0.06

# Set the open interest volume threshold
OI_THRESHOLD = 50

# Risk free rate for the options greeks and IV calculations
risk_free_rate = 0.01

# Buying power percentage to use for the trade
BUY_POWER_LIMIT = 0.05

# Check account buying power
buying_power = float(trade_client.get_account().buying_power)

# Calculate the limit amount of buying power to use for the trade
buying_power_limit = buying_power * BUY_POWER_LIMIT

# Set the expiration date range for the options
min_expiration = today + timedelta(days=21)
max_expiration = today + timedelta(days=60)

# Get the latest price of the underlying stock
def get_underlying_price(symbol):
    # Get the latest trade for the underlying stock
    underlying_trade_request = StockLatestTradeRequest(symbol_or_symbols=symbol)
    underlying_trade_response = stock_data_client.get_stock_latest_trade(underlying_trade_request)
    return underlying_trade_response[symbol].price

# Get the latest price of the underlying stock
underlying_price = get_underlying_price(underlying_symbol)

# Set the minimum and maximum strike prices based on the underlying price
min_strike = str(underlying_price * (1 - STRIKE_RANGE))
max_strike = str(underlying_price * (1 + STRIKE_RANGE))

# Define the criteria for selecting the options
# Each key corresponds to a leg and maps to a tuple of: (expiration range, IV range, delta range, vega range)
criteria = {
    'short_put':  ((21, 60), (0.20, 0.50), (-0.60, -0.10), (0.01, 0.20)),
    'long_put': ((21, 60), (0.20, 0.50), (-0.70, -0.20), (0.01, 0.20))
}

# Set target profit levels
TARGET_PROFIT_PERCENTAGE = 0.4
DELTA_STOP_LOSS = -0.50
IV_STOP_LOSS = 0.40

# Display the values
print(f"Underlying Symbol: {underlying_symbol}")
print(f"{underlying_symbol} price: {underlying_price}")
print(f"Strike Range: {STRIKE_RANGE}")
print(f"Buying Power Limit Percentage: {BUY_POWER_LIMIT}")
print(f"Risk Free Rate: {risk_free_rate}")
print(f"Account Buying Power: {buying_power}")
print(f"Buying Power Limit: {buying_power_limit}")
print(f"Open Interest Threshold: {OI_THRESHOLD}")
print(f"Minimum Expiration Date: {min_expiration}")
print(f"Maximum Expiration Date: {max_expiration}")
print(f"Minimum Strike Price: {min_strike}")
print(f"Maximum Strike Price: {max_strike}")

# Step 4: Historical Market Data Analysis with its stock chart
Factors To Include In The Strategy

## Stock Price Trends (Bar Chart Analysis)

In [ ]:
# Get the historical data for the underlying stock by symbol and timeframe
# ref. https://alpaca.markets/sdks/python/api_reference/data/option/historical.html
def get_stock_data(underlying_symbol, days=90):
    today = datetime.now(timezone).date()
    req = StockBarsRequest(
        symbol_or_symbols=[underlying_symbol],
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),     # specify timeframe
        start=today - timedelta(days=days),                          # specify start datetime, default=the beginning of the current day.
    )
    return stock_data_client.get_stock_bars(req).df

# List of stock agg objects while dropping the symbol column
priceData = get_stock_data(underlying_symbol, days=180).reset_index(level='symbol', drop=True)

import plotly.graph_objects as go

# Bar chart for the stock price
fig = go.Figure(data=[go.Candlestick(x=priceData.index,
                open=priceData['open'],
                high=priceData['high'],
                low=priceData['low'],
                close=priceData['close'])])

fig.show()

## Relative Volatility Index (RVI) for Volatility Directions and Its Degree

In [ ]:
def calculate_rvi(df, period):
    # Calculate daily price changes 
    df['price_change'] = df['close'].diff()

    # Separate up and down price changes
    df['up_change'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['down_change'] = -df['price_change'].where(df['price_change'] < 0, 0)

    # Calculate std of up and down changes over the rolling period
    df['avg_up_std'] = df['up_change'].rolling(window=period).std()
    df['avg_down_std'] = df['down_change'].rolling(window=period).std()

    # Calculate RVI
    df['rvi'] = 100 - (100 / (1 + (df['avg_up_std'] / df['avg_down_std'])))

    # Smooth the RVI with a 4 periods moving average
    df['rvi_smoothed'] = df['rvi'].rolling(window=4).mean()

    return df['rvi_smoothed']

# smoothed RVI
calculate_rvi(priceData, period=21).iloc[30:]

## Bollinger Bands for Volatility Assessment

In [ ]:
# setup bollinger band calculations
def check_bb(df, period=14, multiplier=2):
    bollinger_bands = []
    # Calculate the Simple Moving Average (SMA)
    df['SMA'] = df['close'].rolling(window=period).mean()
    # Calculate the rolling standard deviation
    df['StdDev'] = df['close'].rolling(window=period).std()
    # Calculate the Upper Bollinger Band (two standard deviation)
    df['Upper Band'] = df['SMA'] + (multiplier * df['StdDev'])
    # Calculate the Lower Bollinger Band (two standard deviation)
    df['Lower Band'] = df['SMA'] - (multiplier * df['StdDev'])
    # Get the most recent Upper Band value
    upper_bollinger_band = df['Upper Band'].iloc[-1]
    lower_bollinger_band = df['Lower Band'].iloc[-1]
    
    bollinger_bands = [upper_bollinger_band, lower_bollinger_band]
    return bollinger_bands

bollinger_bands = check_bb(priceData, 14, 2)

# The current market price is not too close to the two-standard deviation level yet but is relatively closer to the higher Bollinger Band.
print(f"Latest Upper Bollinger Band is: {bollinger_bands[0]}. Latest Lower Bollinger Band is {bollinger_bands[1]}; while underlying stock '{underlying_symbol}' price is {underlying_price}.")

# Step 5: Setting Up For A Bear Put spread Options Strategy

Find both short put and long put options.

In [ ]:
# Configure logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
# option_type: ContractType.CALL or ContractType.PUT.
def get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, option_type):
    req = GetOptionContractsRequest(
        underlying_symbols=[underlying_symbol],
        status=AssetStatus.ACTIVE,
        type=option_type,
        strike_price_gte=min_strike,
        strike_price_lte=max_strike,
        expiration_date_gte=min_expiration,
        expiration_date_lte=max_expiration,        
    )
    return trade_client.get_option_contracts(req).option_contracts


In [ ]:
def validate_sufficient_OI(option_data, OI_THRESHOLD):
    '''Ensure that the option has the required fields and sufficient open interest.'''
    if option_data.open_interest is None or option_data.open_interest_date is None:
        return False
    if float(option_data.open_interest) <= OI_THRESHOLD:
        return False
    return True

In [ ]:
def calculate_option_metrics(option_data, underlying_price, risk_free_rate):
    """
    Calculate key option metrics including option price, implied volatility (IV), and option Greeks.
    """
    # Retrieve the latest quote for the option
    option_quote_req = OptionLatestQuoteRequest(symbol_or_symbols=option_data['symbol'])
    option_quote = option_historical_data_client.get_option_latest_quote(option_quote_req)[option_data['symbol']]
    option_price = (option_quote.bid_price + option_quote.ask_price) / 2

    # Calculate expiration and remaining days
    expiration_date = pd.Timestamp(option_data['expiration_date'])
    remaining_days = (expiration_date - pd.Timestamp.now()).days

    # Calculate implied volatility
    iv = calculate_implied_volatility(
        option_price=option_price,
        S=underlying_price,
        K=float(option_data['strike_price']),
        T=max(remaining_days / 365, 1e-6),
        r=risk_free_rate,
        option_type=option_data['type'].value
    )
    
    # Calculate Greeks (delta and vega)
    delta, gamma, theta, vega = calculate_greeks(
        option_price=option_price,
        strike_price=float(option_data['strike_price']),
        expiration=expiration_date,
        underlying_price=underlying_price,
        risk_free_rate=risk_free_rate,
        option_type=option_data['type'].value
    )
    
    return {
        'option_price': option_price,
        'expiration_date': expiration_date,
        'remaining_days': remaining_days,
        'iv': iv,
        'delta': delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega
    }

In [ ]:
def ensure_dict(option_data):
    """
    Convert option_data to a dict using model_dump() if available (for Pydantic models),
    otherwise return the data as-is.
    """
    if hasattr(option_data, "model_dump"):
        return option_data.model_dump()
    return option_data

In [ ]:
def build_option_dict(option_data, underlying_price, risk_free_rate):
    """
    Build an option dictionary by merging raw option data with calculated metrics.
    """
    option_data = ensure_dict(option_data)  # Convert to dict if necessary
    metrics = calculate_option_metrics(option_data, underlying_price, risk_free_rate)
    candidate = {
        'id': option_data['id'],
        'name': option_data['name'],
        'symbol': option_data['symbol'],
        'strike_price': option_data['strike_price'],
        'root_symbol': option_data['root_symbol'],
        'underlying_symbol': option_data['underlying_symbol'],
        'underlying_asset_id': option_data['underlying_asset_id'],
        'close_price': option_data['close_price'],
        'close_price_date': option_data['close_price_date'],
        'expiration_date': metrics['expiration_date'],
        'remaining_days': metrics['remaining_days'],
        'open_interest': option_data['open_interest'],
        'open_interest_date': option_data['open_interest_date'],
        'size': option_data['size'],
        'status': option_data['status'],
        'style': option_data['style'],
        'tradable': option_data['tradable'],
        'type': option_data['type'],
        'initial_IV': metrics['iv'],
        'initial_delta': metrics['delta'],
        'initial_gamma': metrics['gamma'],
        'initila_theta': metrics['theta'],
        'initial_vega': metrics['vega'],
        'initial_option_price': metrics['option_price'],
    }
    
    return candidate

In [ ]:
def check_candidate_option_conditions(candidate, criteria, label):
    """
    Check whether a candidate option meets the filtering criteria.
    The criteria is a tuple of (expiration_range, iv_range, delta_range, vega_range).
    Logs detailed information if a candidate fails a criterion.
    """
    expiration_range, iv_range, delta_range, vega_range = criteria

    if not (expiration_range[0] <= candidate['remaining_days'] <= expiration_range[1]):
        logger.debug(f"{candidate['symbol']} fails expiration condition for {label}: remaining_days {candidate['remaining_days']} not in {expiration_range}.")
        return False
    if not (iv_range[0] <= candidate['initial_IV'] <= iv_range[1]):
        logger.debug(f"{candidate['symbol']} fails IV condition for {label}: initial_IV {candidate['initial_IV']} not in {iv_range}.")
        return False
    if not (delta_range[0] <= candidate['initial_delta'] <= delta_range[1]):
        logger.debug(f"{candidate['symbol']} fails delta condition for {label}: initial_delta {candidate['initial_delta']} not in {delta_range}.")
        return False
    if not (vega_range[0] <= candidate['initial_vega'] <= vega_range[1]):
        logger.debug(f"{candidate['symbol']} fails vega condition for {label}: initial_vega {candidate['initial_vega']} not in {vega_range}.")
        return False

    return True

In [ ]:
def pair_put_candidates(short_puts: List[Dict[str, Any]], long_puts: List[Dict[str, Any]], underlying_price: float) -> Tuple[Optional[Dict[str, Any]], Optional[Dict[str, Any]]]:
    """
    For the bear put spread, require: short_put strike <= underlying_price < long_put strike.
    Returns the first valid pair found.
    """
    for sp in short_puts:
        for lp in long_puts:
            if sp['expiration_date'] == lp['expiration_date'] and sp['strike_price'] <= underlying_price < lp['strike_price']:
                logger.info(f"Selected Bear Put spread: short_put {sp['symbol']} and long_put {lp['symbol']} with expiration {sp['expiration_date']}.")
                return sp, lp
    
    
    # If no valid pair is found, log the expiration date (if available) from the candidate lists.
    expiration_info = None
    if short_puts:
        expiration_info = short_puts[0]['expiration_date']
    elif long_puts:
        expiration_info = long_puts[0]['expiration_date']

    if expiration_info:
        logger.info(f"No valid bear put spread pair found for expiration {expiration_info} with the given candidates and underlying price conditions.")
    else:
        logger.info("No valid bear put spread pair found: no candidate data available.")
    
    return None, None

In [ ]:
def check_buying_power(short_put: Dict[str, Any], long_put: Dict[str, Any], buying_power_limit: float) -> None:
    """
    Calculates the total premium paid (risk) for a bear put spread and checks it against the buying power limit.
    If the buying power requirement is not met, the exception is thrown and the rest of the code is never executed.
    """
    option_size = float(short_put['size'])
    risk = (long_put['initial_option_price'] - short_put['initial_option_price']) * option_size
    logger.info(f"Calculated bear put spread risk: {risk}.")
    
    if risk >= buying_power_limit:
        raise Exception('Buying power limit exceeded for a bear put spread risk.')

In [ ]:
def find_options_for_bear_put_spread(put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD):
    """
    Orchestrates the workflow to build a bear put spread.
    Groups options by expiration, filters them with criteria, pairs candidates using helper functions,
    and checks buying power.
    
    Returns:
        A list of legs [short_put, long_put] if a valid pair is found, or an empty list otherwise.
    """
    short_put_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}
    long_put_candidates_by_exp: Dict[pd.Timestamp, List[Dict[str, Any]]] = {}

    # Process each option candidate
    for option_data in put_options:
        if not validate_sufficient_OI(option_data, OI_THRESHOLD):
            logger.warning(f"Insufficient open interest for option {getattr(option_data, 'symbol', 'unknown')} (threshold: {OI_THRESHOLD}). Skipping candidate.")
            continue

        candidate = build_option_dict(option_data, underlying_price, risk_free_rate)
        expiration_date = candidate['expiration_date']
        short_put_candidates_by_exp.setdefault(expiration_date, [])
        long_put_candidates_by_exp.setdefault(expiration_date, [])
        
        # Check each candidate for both put criteria
        if check_candidate_option_conditions(candidate, criteria['short_put'], 'short_put'):
            short_put_candidates_by_exp[expiration_date].append(candidate)
            logger.info(f"Added {candidate['symbol']} as a short put candidate for expiration {expiration_date}.")
        if check_candidate_option_conditions(candidate, criteria['long_put'], 'long_put'):
            long_put_candidates_by_exp[expiration_date].append(candidate)
            logger.info(f"Added {candidate['symbol']} as a long put candidate for expiration {expiration_date}.")

    # Process only expiration dates common to both candidate groups
    common_expirations = set(short_put_candidates_by_exp.keys()) & set(long_put_candidates_by_exp.keys())
    for expiration_date in common_expirations:
        sp, lp = pair_put_candidates(short_put_candidates_by_exp[expiration_date],
                                      long_put_candidates_by_exp[expiration_date],
                                      underlying_price)
        if sp and lp:
            try:
                check_buying_power(sp, lp, buying_power_limit)
            except Exception as e:
                logger.error(f"Pair for expiration {expiration_date} failed buying power check: {e}")
                continue
            logger.info(f"Selected bear put spread for expiration {expiration_date}: short {sp['symbol']}, long {lp['symbol']}.")
            return [sp, lp]

    logger.info("No valid bear put spread found.")
    return None, None

In [ ]:
put_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, ContractType.PUT)

In [ ]:
sp, lp = find_options_for_bear_put_spread(put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)

# Step 6: Executing A Bear Put Spread Options Strategy

In [ ]:
def place_bear_put_spread_order(short_put, long_put):
    """
    Place a bear put spread order if both short_put and long_put data are provided.
    """
    if not (short_put and long_put):
        logger.info("No valid bear put spread found.")
        return None

    try:
        # Build order legs: sell the short put and buy the long put.
        order_legs = [
            OptionLegRequest(
                symbol=short_put['symbol'],
                side=OrderSide.SELL,
                ratio_qty=1
            ),
            OptionLegRequest(
                symbol=long_put['symbol'],
                side=OrderSide.BUY,
                ratio_qty=1
            )
        ]

        # Create a market order for a multi-leg (spread) order.
        req = MarketOrderRequest(
            qty=1,
            order_class=OrderClass.MLEG,
            time_in_force=TimeInForce.DAY,
            legs=order_legs
        )
        res = trade_client.submit_order(req)
        logger.info("A bear put spread order placed successfully.")
        return res
    except Exception as e:
        logger.error(f"Failed to place a bear put spread order: {e}")
        return None

In [ ]:
res = place_bear_put_spread_order(sp, lp)
res

# Step 7: Rinse or Roll the Option (Optional)

In [ ]:
def roll_rinse_bear_put_spread(short_put, long_put, rolling, target_profit_percentage, delta_stop_loss_thres, iv_stop_loss_thres, option_type, criteria, risk_free_rate, min_strike, max_strike, min_expiration, max_expiration, OI_THRESHOLD):
    """
    Checks if a bear put spread meets exit criteria (profit or stop-loss levels)
    based on current option price, delta, and vega. If the criteria are met,
    it closes the existing spread and, if rolling=True, attempts to open a new spread.
    Returns:
        Tuple containing a status message and new spread data if a new spread is opened, otherwise None.
    """
    underlying_symbol = short_put['underlying_symbol']
    underlying_price = get_underlying_price(underlying_symbol)
    
    # Retrieve current quote for the short put
    option_symbol = short_put['symbol']
    option_quote_request = OptionLatestQuoteRequest(symbol_or_symbols=option_symbol)
    option_quote = option_historical_data_client.get_option_latest_quote(option_quote_request)[option_symbol]
    current_short_price = (option_quote.bid_price + option_quote.ask_price) / 2
    
    metrics = calculate_option_metrics(short_put, underlying_price, risk_free_rate)
    current_delta = metrics['delta']
    current_iv = metrics['iv']
    
    # Determine target exit price based on the initial premium received for the short put.
    target_price = metrics['option_price'] * target_profit_percentage
    
    # Check exit criteria: either the short put price is at or below the target,
    # the absolute delta fall short of the threshold, or iv is above the threshold.
    if current_short_price <= target_price or abs(current_delta) <= delta_stop_loss_thres or current_iv >= iv_stop_loss_thres:
        logger.info(f"Exit criteria met for {underlying_symbol}: current_short_price={current_short_price}, "
                    f"target_price={target_price}, delta={current_delta}, iv={current_iv}.")
        # Execute the roll or rinse (exit) of the spread
        try:
            # Close the short put
            trade_client.close_position(
                symbol_or_asset_id=short_put['symbol'],
                close_options=ClosePositionRequest(qty='1')
            )
            logger.info(f"Closed short put: {short_put['symbol']}")
            
            # Close the long put
            trade_client.close_position(
                symbol_or_asset_id=long_put['symbol'],
                close_options=ClosePositionRequest(qty='1')
            )
            logger.info(f"Closed long put: {long_put['symbol']}")
        except Exception as e:
            msg = f"Failed to close existing bear put spread on {underlying_symbol}: {e}"
            logger.error(msg)
            return msg, None

        # If rolling, attempt to open a new bear put spread
        if rolling:
            try:
                # Find latest put options
                put_options = get_options(underlying_symbol, min_strike, max_strike, min_expiration, max_expiration, option_type)
                # Find new bear put spread candidates
                sp, lp = find_options_for_bear_put_spread(put_options, underlying_price, risk_free_rate, buying_power_limit, criteria, OI_THRESHOLD)
                # Place a new bear put spread order and return the response message
                res = place_bear_put_spread_order(sp, lp)
                if res:
                    new_spread = {
                        'short_put': sp,
                        'long_call': lp
                    }
                    return f"Rolled bear put spread on {underlying_symbol}. {res}", new_spread
                else:
                    msg = f"Failed to open new bear put spread on {underlying_symbol} after closing."
                    logger.error(msg)
                    return msg, None
            except Exception as e:
                msg = f"Failed to roll into a new bear put spread on {underlying_symbol}: {e}"
                logger.error(msg)
                return msg, None
        else:
            # If not rolling, simply exit the position
            return f"Closed (rinsed) bear put spread on {underlying_symbol}.", None
    else:
        # Criteria not met; hold the position.
        msg = (f"Holding bear put spread on {underlying_symbol}: current_short_price={current_short_price}, "
               f"target_price={target_price}, delta={current_delta}, vega={current_iv}.")
        logger.info(msg)
        return msg, None


In [ ]:
message, new_spread = roll_rinse_bear_put_spread(sp, lp, True, TARGET_PROFIT_PERCENTAGE, DELTA_STOP_LOSS, IV_STOP_LOSS, ContractType.CALL, criteria, risk_free_rate, min_strike, max_strike, min_expiration, max_expiration, OI_THRESHOLD)